Notebook to retile the mosaic predictions into larger tiles to reduce tiling effects on cutting crop fields, do thresholding to produce binarised crop boundary, masking noncrop fields and vectorise results.

## Load packages and modules

In [1]:
import numpy as np
import imageio.v2 as imageio
from osgeo import gdal
import os
from glob import glob
from skimage import measure, util
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union
import rasterio
from deafrica_tools.spatial import xr_rasterize
import subprocess
import sys
from shapely.geometry import box
import rioxarray
import datacube
from tqdm import tqdm
module_paths=['../1_Identify_months_thresholds_model_evaluation']
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


## Define parameters

In [2]:
from datasets import export_geotiff

In [3]:
tiles_shp='input_data/Rwanda_tiles_edited.shp'
extent_mosaic='results/processed/Rwanda_extent_prob_2021_04_10_12_mosaic.tif'
bound_mosaic='results/processed/Rwanda_bound_prob_2021_04_10_12_mosaic.tif'
instance_mosaic='results/processed/Rwanda_field_instance_2021_04_10_12_mosaic.tif'
grd_search_df='../1_Identify_months_thresholds_model_evaluation/results/averaged/Rwanda_grid_search_thresholds.csv' # grid search results of thresholds
country = 'Rwanda'
str_year='2021'
out_crs='epsg:3857'

In [4]:
out_folder='results_retiled'
if not os.path.isdir(out_folder):
    os.makedirs(out_folder)

## Read in tiles

In [5]:
tiles=gpd.read_file(tiles_shp).to_crs(out_crs)
bboxes=tiles.bounds
crs=tiles.crs.to_string()
tiles

,id,left,top,right,bottom,geometry
0,2.0,706963.175735,9.834132e+06,756963.175735,9.784132e+06,"POLYGON ((3241149.768 -176033.989, 3262787.060..."
1,3.0,706963.175735,9.784152e+06,756963.175735,9.734152e+06,"POLYGON ((3212755.055 -217315.182, 3262770.401..."
2,4.0,706963.175735,9.734172e+06,756963.175735,9.684172e+06,"POLYGON ((3212816.801 -267661.192, 3262847.026..."
3,5.0,756943.175735,9.884112e+06,806943.175735,9.834112e+06,"POLYGON ((3262655.373 -149123.099, 3312674.875..."
4,6.0,756943.175735,9.834132e+06,806943.175735,9.784132e+06,"POLYGON ((3262689.705 -166934.480, 3312677.348..."
5,7.0,756943.175735,9.784152e+06,806943.175735,9.734152e+06,"POLYGON ((3262750.397 -217252.774, 3312749.809..."
6,8.0,756943.175735,9.734172e+06,806943.175735,9.684172e+06,"POLYGON ((3262827.017 -267584.296, 3312841.286..."
7,9.0,806923.175735,9.884112e+06,856923.175735,9.834112e+06,"POLYGON ((3312560.322 -140482.917, 3362594.486..."
8,10.0,806923.175735,9.834132e+06,856923.175735,9.784132e+06,"POLYGON ((3312657.357 -166876.243, 3362626.000..."
9,11.0,806923.175735,9.784152e+06,856923.175735,9.734152e+06,"POLYGON ((3312729.813 -217176.959, 3362710.203..."


### thresholding the boundary probability layer
* clip boundary and extent probabilities mosaics to tile extent
* thresholding on the boundary and extent tiles
* label the binary tiles to generate individual crop fields
* filter regions with few crop pixels using DE Africa crop extent layer
* polygonise and export masked fields in vector and raster formats
* remove intermediate files

In [6]:
t_bnd=0.75 # boundaring probability threshold
t_extent=0.5 # extent probability threshold
t_wofs=0.5 # threshold wet frequency for wofs
t_pct_crop=0.1 # threshold percentage of crop pixels within field object

In [ ]:
shp_tiles_unmasked=[]
shp_tiles_masked_bound=[]
shp_tiles_masked_exent=[]
for index,tile in tiles.iterrows():
    print('processing tile ',index)
    # get bbox
    minx,miny,maxx,maxy=bboxes.iloc[index]
    
    # clip predictions mosaic using tile
    print('retiling the boundary and extent rasters...')
    
    out_bound=os.path.join(out_folder, country+'_bound_prob_tile_'+str(index)+'.tif')
    if os.path.exists(out_bound):
        print('tiled boundary file exists, skipping...')
    else:
        gdal_cmd=["gdal_translate", "-of", "GTiff",
              "-projwin",str(minx),str(maxy),str(maxx),str(miny),
              '-projwin_srs',crs,bound_mosaic,out_bound]
        subprocess.run(gdal_cmd,stdout=subprocess.DEVNULL)
    
    out_extent=out_bound.replace('bound','extent')
    if os.path.exists(out_extent):
        print('tiled extent file exists, skipping...')
    else:
        gdal_cmd=["gdal_translate", "-of", "GTiff",
          "-projwin",str(minx),str(maxy),str(maxx),str(miny),
          '-projwin_srs',crs,extent_mosaic,out_extent]
        subprocess.run(gdal_cmd,stdout=subprocess.DEVNULL)
    
    out_instance=out_bound.replace('bound','field_instance')
    if os.path.exists(out_instance):
        print('tiled instance file exists, skipping...')
    else:
        gdal_cmd=["gdal_translate", "-of", "GTiff",
          "-projwin",str(minx),str(maxy),str(maxx),str(miny),
          '-projwin_srs',crs,instance_mosaic,out_instance]
        subprocess.run(gdal_cmd,stdout=subprocess.DEVNULL)
        
    # read in clipped predictions
    ds_extent = gdal.Open(out_extent)
    geotrans=ds_extent.GetGeoTransform()
    proj=ds_extent.GetProjection()
    ds_extent=None

    extent_prob=imageio.imread(out_extent)
    bound_prob=imageio.imread(out_bound)
    instance=imageio.imread(out_instance)
    
    # do thresholding and masking out background
    print('thresholding on boundary probability...')
    bound_binary=bound_prob<t_bnd
    bound_binary[bound_prob==0]=0
    bound_binary[instance==0]=0
    
    print('thresholding on extent probability...')
    extent_binary=extent_prob>=t_extent
    extent_binary[extent_prob==0]=0
    extent_binary[instance==0]=0
    
    # label connected regions, non-field will be labelled as 0
    binary_extent_labelled,n_features_extent= measure.label(extent_binary, background=0,return_num=True)
    print('number of fields from thresholded extent probabilities: ',n_features_extent)
    binary_bound_labelled,n_features_bound= measure.label(bound_binary, background=0,return_num=True)
    print('number of fields from thresholded boundary probabilities: ',n_features_bound)

    # query DE Africa crop mask layer
    xr_ds=rioxarray.open_rasterio(out_bound).to_dataset(name='binarised boundary')
    dc = datacube.Datacube(app='cropland_extent')
    cm = dc.load(product='crop_mask',measurements=['filtered'],like=xr_ds,time=('2019'),resampling='nearest')
    np_crop_mask=cm['filtered'].squeeze().to_numpy()
    np_crop_mask=np_crop_mask==1
    
    # query wofs layer
    dc = datacube.Datacube(app='water_extent')
    wofs = dc.load(product="wofs_ls_summary_annual",measurements=['frequency'],like=xr_ds,time=('2019'),resampling='nearest')
    water_mask=wofs['frequency'].squeeze().to_numpy()
    water_mask=water_mask<t_wofs
    
    # combined mask of crop exent and wofs
    overall_mask=(np_crop_mask&water_mask)
    # filter using DE Africa crop mask and wofs layer
    print('masking binary boundary layer')
    # table is a dictionary mapping column names to data columns
    # (NumPy arrays)
    table = measure.regionprops_table(
        binary_bound_labelled,
        overall_mask,
        properties=('label','intensity_mean'),
    )
    condition = table['intensity_mean']>t_pct_crop
    # zero out labels not meeting condition
    input_labels = table['label']
    output_labels = input_labels * condition
    binary_bound_labelled_masked = util.map_array(
        binary_bound_labelled, input_labels, output_labels
    )
            
    print('masking binary extent layer')
    table = measure.regionprops_table(
        binary_extent_labelled,
        overall_mask,
        properties=('label','intensity_mean'),
    )
    condition = table['intensity_mean']>t_pct_crop
    # zero out labels not meeting condition
    input_labels = table['label']
    output_labels = input_labels * condition
    binary_extent_labelled_masked = util.map_array(
        binary_extent_labelled, input_labels, output_labels
    )
        
    # convert back to binary layers
    binary_extent_masked=binary_extent_labelled_masked>0
    binary_bound_masked=binary_bound_labelled_masked>0
    
    # export masked as geotiff
    outname_bound_binary_masked=os.path.join(out_folder,os.path.basename(bound_mosaic).replace('mosaic','masked_tile_')[:-4]+str(index)+'.tif')
    outname_bound_binary_masked=outname_bound_binary_masked.replace('bound','binarised_bound')
    outname_extent_binary_masked=outname_bound_binary_masked.replace('bound','extent')
    export_geotiff(outname_bound_binary_masked,binary_bound_masked,geotrans,proj,gdal.GDT_Byte)
    export_geotiff(outname_extent_binary_masked,binary_extent_masked,geotrans,proj,gdal.GDT_Byte)

    # polygonise masked
    print('polygonizing filtered binarised boundary probability...')
    outname_shp_boundary_masked=outname_bound_binary_masked.replace('.tif','.shp')
    shp_tiles_masked_bound.append(outname_shp_boundary_masked)
    if os.path.exists(outname_shp_boundary_masked):
        print('file existing, skipping...')
    else:
        cmd=['gdal_polygonize.py','-8','-mask',outname_bound_binary_masked,'-b','1',outname_bound_binary_masked,outname_shp_boundary_masked]
        subprocess.run(cmd)
        
    print('polygonizing filtered binarised extent probability...')
    outname_shp_extent_masked=outname_extent_binary_masked.replace('.tif','.shp')
    shp_tiles_masked_exent.append(outname_shp_extent_masked)
    if os.path.exists(outname_shp_extent_masked):
        print('file existing, skipping...')
    else:
        cmd=['gdal_polygonize.py','-8','-mask',outname_extent_binary_masked,'-b','1',outname_extent_binary_masked,outname_shp_extent_masked]
        subprocess.run(cmd)
    
    # remove unwanted files
    os.remove(out_bound)
    os.remove(out_extent)
    os.remove(out_instance)

### merge tiled vectors

In [ ]:
! ogrmerge.py -o results_retiled/Rwanda_binarised_bound_prob_2021_04_10_12_masked_tiles_merged.shp -single results_retiled/Rwanda_binarised_bound_prob_2021_04_10_12_masked_tile_*.shp

In [ ]:
! ogrmerge.py -o results_retiled/Rwanda_binarised_extent_prob_2021_04_10_12_masked_tiles_merged.shp -single results_retiled/Rwanda_binarised_extent_prob_2021_04_10_12_masked_tile_*.shp